<a href="https://colab.research.google.com/github/fujitako03/machine-learning-tutorials/blob/main/OpenAI_text_davinci_003_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open AIのChatGPT（text-davinci-003）を使ってみる
[公式サイト](https://beta.openai.com/examples/default-friend-chat)

In [ ]:
!pip install openai

In [2]:
class Config:
  openai_api_key="<API_KEY>"

## とりあえず動かしてみる
サンプルコードをそのまま採用

In [5]:
import openai

openai.api_key = Config.openai_api_key

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="You: What have you been up to?\nFriend: Watching old movies.\nYou: Did you watch anything interesting?\nFriend:",
  temperature=0.5,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=["You:"]
)

In [12]:
response["choices"][0]["text"]

' Yeah, I watched a classic western called The Good, the Bad and the Ugly. It was really good!'

## パラメータの意味を理解する

In [17]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt="You: What have you been up to?\nFriend: Watching old movies.\nYou: Did you watch anything interesting?\nFriend:",
  temperature=0, # ランダム性
  max_tokens=20,# 最大単語数
  top_p=1.0, # ランダム性
  frequency_penalty=0.5, # 頻出のペナルティ
  presence_penalty=0.0 # 同じ話題のペナルティ
  stop=["You:"]
)
response["choices"][0]["text"]

' Yeah, I watched The Godfather. It was really good.'

### temperature(デフォルト1)
0から2の範囲を取り、出力する単語のランダム性を指定する
0であれば毎回同じ文章を生成します（分類タスクに適している）。2であれば完全にランダムな文章になります。その中間(ドキュメントでは0.9を推奨)を選ぶことで適度にランダムで意味の通った文章を生成することが可能です。

top_pとの同時変更は推奨しない

### top_p 
top_pを0.1にすると、予測する次の単語は確率の高い上位10%の候補から選択されます。小さくすればするほど候補が確率の高いものに絞られるため、確定的になっていきます。大きくすれば色んな単語から選ぶようになるので、よりランダム性が強くなります。

temperatureとの同時変更はお勧めしない

### max_tokens
最大単語数

### presence_penalty
2.0 から 2.0 の間の数値。正の値は、新しいトークンがこれまでのテキストに現れるかどうかに基づいてペナルティを課し、新しいトピックについて話すモデルの可能性を高める。

### frequency_penalty
2.0 から 2.0 の間の数。正の値は、これまでのテキストにおける既存の頻度に基づいて新しいトークンにペナルティを課し、モデルが同じ行をそのまま繰り返す可能性を低下させる。

### stop
それ以上の生成を停止する最大4つのシーケンス


## temperatureを変えて実行してみる

In [28]:
temperature_samples = [0, 0.5, 1, 1.5, 2]
for temperature in temperature_samples:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="私: どんな映画が好き？私はハリーポッターみたいなファンタジー系が好きかな。\n友達:",
    temperature=temperature,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=0.0,
    stop=["私:"]
  )
  print(temperature, response["choices"][0]["text"])

0  私もハリーポッターが好きです！私はアクション映画も好きです。
0.5  私は、サスペンス系の映画が好きです。また、コメディー映画も好きです。
1  私もハリーポッターの映画が好きです。他にはSFの物語が好きです。
1.5  いや、私も同じですね! こだわりの、乞食ハサールも超絶個性的ジャンルがtopです！ある
2  あーそれ好きだ！立


1.5以上はかなりおかしい

## top_pを変えて実行してみる

In [31]:
top_p_samples = [0, 0.25, 0.5, 0.75 1]
for top_p in top_p_samples:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="私: どんな映画が好き？私はハリーポッターみたいなファンタジー系が好きかな。\n友達:",
    temperature=1,
    max_tokens=60,
    top_p=top_p,
    frequency_penalty=0.5,
    presence_penalty=0.0,
    stop=["私:"]
  )
  print(top_p, response["choices"][0]["text"])

0  私もハリーポッターが好きです！私は、アクション映画やコメディーも好きです。
0.25  私もハリーポッターが好きです！私は、アクション映画やコメディーも好きです。
0.5  私もハリーポッターは大好きです！他には、スリラーやコメディなども好きです。
0.75  私もそう思うよ！僕はアクション系が好きだよ。
1  私はコメディやサスペンスの映画が好きです。今日映画館に行ったとき、話題のサスペンスを


## presence_penaltyを変えて実行してみる

In [33]:
presence_penalty_samples = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]
for presence_penalty in presence_penalty_samples:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="私: 今日も運動したよ！\n友達: すごいね！これで2日連続だね\私: ありがとう！あなたのおかげだね\n友達:",
    temperature=1,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=presence_penalty,
    stop=["私:"]
  )
  print(presence_penalty, response["choices"][0]["text"])

-2  勇気をくれてありがとう。
-1.5  あなたが気合が入っていたから大丈夫だよ！
-1  いえ、当然だよ！お互い頑張ろう！
-0.5  いや、私は何もしてないよ！あなたが頑張ったんだからね！
0  うん、自分の目標につながっているのはうれしいよ！
0.5  いえ、君のおかげだよ！勇気をもらって頑張れたんだから
1  全然大したことじゃないよ！
1.5  嬉しい！やる気を保つためにも、お互い頑張りましょう！
2  いや、あなたの努力のおかげだよ！


±1.5以上の極端じゃなければわりと自然。0以上の人のほうが会話してて楽しそう

In [34]:
presence_penalty_samples = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]
for presence_penalty in presence_penalty_samples:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="私: どんな映画が好き？私はハリーポッターみたいなファンタジー系が好きかな。\n友達:",
    temperature=1,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=presence_penalty,
    stop=["私:"]
  )
  print(presence_penalty, response["choices"][0]["text"])

-2  私もファンタジー系が好きで、スターウォーズも好きです。ルーファスやザ・ウォーズ・オブ・ザ・リングも、『インフ
-1.5  私もハリーポッターの映画は好きだよ！僕は、よりリアルな映画も好きなんだ。例えば、実
-1  私はアクション系が好みですが、自分で行動するという設定がある映画はとても好きです。大
-0.5  私もです！スターウォーズやマーベルなども好きですよ。
0  私もハリーポッターが大好きです！ただ、アクション、スリラーなども好きです。
0.5  私もハリーポッターが大好きです！私は面白いコメディ映画やサスペンスも好きです。
1  私は現実の世界を描いた映画が好きです。例えば、ソーシャルメディアやインターネットに関するも
1.5  そうなんだ！私もハリーポッターが大好きなんだよ。他にはアクション系とかスリラー映画も好きなの
2  僕もファンタジー系が好きですよ！例えば、ライオンキングのようなアニメ映画なんかも面白い


- 1以上はコメディーも推してきた。数うちゃあたる戦法か？
- 友達だから肯定してくれるのは確定なのか？本当の友達は、、、
- -2、2みたいな極端な値はやや表現が不自然な印象

## frequency_penaltyをいろいろ変えて実行してみる

In [26]:
frequency_penalty_samples = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]
for frequency_penalty in frequency_penalty_samples:
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="私: どんな映画が好き？私はハリーポッターみたいなファンタジー系が好きかな。\n友達:",
    temperature=1,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=frequency_penalty,
    presence_penalty=0.0,
    stop=["私:"]
  )
  print(frequency_penalty, response["choices"][0]["text"])

-2  うん！私もハリーポッターは大好きなんだ！私も私はダイアリー！スリー！！！！
-1.5  映画もそうだけど、なぜそのジャンルがおおおおおおおおおおおおおおおおお
-1  そうなんだ！私もスター・ウォーズみたいなスペースオペラ系が好きなんだ！
-0.5  私も上記と同じくハリーポッターもすごく好きです。でも私もSFやサスペンスも好きです。
0  うん、私もハリーポッターが大好き！大人の私にも全く新鮮な感覚だな。スターウォーズやマーベルも
0.5  私もハリーポッターが大好きだよ。私はヒーロー系の映画も好き。アベンジャーズやバットマン、スパイダ
1  うん、私もだよ！その他にサスペンスやコメディ映画も好きなんだ。ハリーポッターは子供と大人に
1.5  私も同じように思います！他に映画を楽しむことは何でもいいです。アクション系、コメディー
2  私もハリーポッターが好きで、サイコパスなどのミステリー系映画も面白いと思うよ。


-1.5以下はかなりおかしなことに。無難に0か0.5にしておくのがよさそう